### Introduction

Nowadays, unsuitable news content for under 12-year-old children is increasing in Taiwan. Those violent, bloody, and pornographic content should be restricted. To protect these young audiences' mental health, I build a deep learning model that can avoid them from reaching these types of news. In the project, I build the RNN(Recurrent Neural Network) model with Python, and every detailed step is below. 



In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot
import matplotlib.pyplot as plt


In [ ]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

### 01. Import text data 

1.   First, using Python to crawl news websites and grab the news headlines.
2.   Artificially mark all news headlines as "0" and "1". (0 refers to the warning; 1 refers to allow to read)
3.   Save as a CSV file and upload to google drive.

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [ ]:
df = pd.read_csv('NewsTitle2.csv',sep=",")

In [ ]:
df.head()

,新聞標題,是否適宜觀看
0,男偶像自爆「出道之後就沒談過戀愛」 理想型是《寄生上流》的她,1
1,女大生遭男同學打手槍射在身上 校方竟要她「當作沒發生」,0
2,新加坡外籍勞工已15萬染疫 遠超本國公民,1
3,她「5年不給碰」老公偷吃人妻 網挺男方：是被妳逼的,1
4,偏鄉和順國小學童走讀 結合棒棒積木飯店漂書啟動,1


### 02. Keep text only in Chinese

Numbers in news headlines do not affect deep learning too much. Use the regular expression (`re`) to remove all the numbers.


In [ ]:
import re

The range for Chinese characters in the Unicode is "u4E00 - u9FD5".

In [ ]:
patn=re.compile(r"[\u4E00-\u9FD5]+")

At the same time, separate the words in all sentences.

In [ ]:
for review in df['新聞標題']:
    print(patn.findall(review))

串流輸出內容已截斷至最後 5000 行。
['新增', '例境外', '男性移工分別自印尼', '菲律賓入境']
['尼日遭恐怖攻擊', '至少', '平民遭槍殺']
['冬令食補', '呷健康養生']
['蔡依林新', '甜秘密', '負評多', '導演廖人帥整夜失眠發文感嘆']
['鹽水月津港公', '上游水岸步道完工', '串連新營鹽水舊水道']
['朱木炎', '公分球星譚傑龍捲桃色詐騙', '刑事局擴大偵辦']
['中研院研究員吳漢忠', '獲選美國國家發明家學院院士']
['明星游擊', '強投一次到位', '大都會總裁', '已經規劃很久']
['與中國關係持續緊張之際', '澳洲內閣改組']
['范雲籲學術資料庫公共化', '國圖', '一直有在做']
['批川普政府疫苗接種進度落後', '拜登', '上任全力追趕']
['道瓊社', '迪士尼董事長表達有意願擔任駐中國大使']
['正妹胸部大到崩潰', '長到', '罩杯還沒停']
['母獅闖水牛群遭', '踩踏凌虐', '圍攻', '拋飛重摔慘死水坑']
['胡宇威骨折坐輪椅力挺', '全明星運動會', '李玖哲不捨道別淚灑小巨蛋']
['疫苗開打近滿月', '疫苗覆蓋率第一名竟是以色列']
['害死擎天崗水牛的不只天氣', '法官判決', '把人民當媽寶', '恐也是幫凶']
['直銷神壇詐信徒', '宗主詐捐款爽買', '豪宅', '信徒不敢退會恐怖理由曝光']
['罐罐難過', '罐罐過', '基隆安中贏得適應體育教案首獎']
['孩子看到他就怕', '機警老師察覺報案揪色狼']
['擎天崗', '水牛暫緩移置', '陽管處', '健康狀況不宜']
['梁靜茹唱', '分手快樂', '哽咽落淚', '笑虧周興哲感情路有不少挫折']
['府城驚艷意外收穫', '夏威夷樂器行']
['在美早產醫藥費近', '萬', '母盼群眾募資援助']
['侏儸紀', '真實版', '男遇', '恐龍襲擊', '全身多處被撕咬']
['直播', '境外增', '例', '陳時中親自說明']
['東京激增', '確診創新高', '菅義偉明發布緊急事態宣言']
['厄文', '杜蘭特關鍵時刻輪流打鐵', '名嘴轟']
['日本央行決議緊急貸放計畫延長半年']
['機師約砲不防疫', '紐籍機師才成防疫缺口', '

After removing numbers, use blank space to connect the words in all sentences.

In [ ]:
x_tmp = []

for review in df['新聞標題']:
    sen_list = patn.findall(review)
    sen = ' '.join(sen_list)
    x_tmp.append(sen)

In [ ]:
x_tmp

['男偶像自爆 出道之後就沒談過戀愛 理想型是 寄生上流 的她',
 '女大生遭男同學打手槍射在身上 校方竟要她 當作沒發生',
 '新加坡外籍勞工已 萬染疫 遠超本國公民',
 '她 年不給碰 老公偷吃人妻 網挺男方 是被妳逼的',
 '偏鄉和順國小學童走讀 結合棒棒積木飯店漂書啟動',
 '少女徹夜未歸與 男網友一夜情 母街頭苦等揭荒誕行徑',
 '論壇 用公投重新守護民主與食安',
 '蘇偉碩遭查水表 民進黨 江啓臣要為造謠行為背書嗎',
 '再紅也沒用 日動畫師錢比超商員工少',
 '支付寶 微信支付入列 川普禁 款中國大陸軟體',
 '人妻逛老街問老菜脯價格 店家狠嗆 不買就滾 去吃加色素的',
 '華藝疑違法配合陸方審查論文 教育部 將專案調查',
 '億點閱 女神降臨 有臉蛋天才 未演先轟動',
 '影 綠鬣蜥過馬路 彰化網友驚呼 蔬菜產區淪陷了嗎',
 '影 基隆封溪 年 移除 種外來魚類共 尾',
 '冬天必吃 全聯 神雞湯 讓她整鍋嗑光 網讚 都是大塊腿肉',
 '首例 英國變種病毒 入侵法國 自倫敦返法無症狀隔離中',
 '寧夏古早味征服饕客 千金傳承美味',
 '防萊豬進校園嘉市執行防萊 步驟 黃敏惠帶校長觀摩',
 '火爆哥騎車 踹一腳 代價萬元又涉恐嚇',
 '歲方志友帶球嫁入楊家 第一天被公公舉動惹淚了',
 '川普支持者闖國會爆衝突 當局證實一警傷重不治',
 '侯友宜點名永和大陳 年內開工 都更拚明年核准 件',
 '莫斯科芭蕾舞團 全部停演',
 '歲翁撞 萬豪車嚇 怎麼賠',
 '群聚病例增加遭多州禁入 澳洲大城雪梨陷孤立',
 '氣溫斷崖式崩跌冷爆 網揭 保暖物',
 '松阪蘇 馬卡茸 薑 母鴨 政壇大咖該補修的庶民學分',
 '爸做忌沒人煮菜 女友 菜 湯感動網',
 '衛福部拍板健保費調漲 蔡英文 集結力量給醫護最大支持',
 '台幣強升 出口導向企業 月賠 萬 憂倒閉潮',
 '房子銀子孩子都被妻掌握 科技男提離婚外遇竟曝光',
 '凌晨 點揪男同學在客廳吃泡麵 女兒 超扯解釋 讓爸起疑',
 '病毒變種 英國變種病毒 流竄 德國也現蹤跡',
 '小米 終於登台 接下來還會引進入門款',
 '古斌憋氣到缺氧 瀕死演技好真實',
 '家裡舊車換成新買的進口車 他曝曖昧女 反差嘴臉 網勸放生',
 '中共黨員新權利條例 海外

### 03. Compute each word's frequency

Put all sentences in a string and count the number of occurrences for each word.

In [ ]:
egg = ''.join(''.join(x_tmp).split())

In [ ]:
egg

'男偶像自爆出道之後就沒談過戀愛理想型是寄生上流的她女大生遭男同學打手槍射在身上校方竟要她當作沒發生新加坡外籍勞工已萬染疫遠超本國公民她年不給碰老公偷吃人妻網挺男方是被妳逼的偏鄉和順國小學童走讀結合棒棒積木飯店漂書啟動少女徹夜未歸與男網友一夜情母街頭苦等揭荒誕行徑論壇用公投重新守護民主與食安蘇偉碩遭查水表民進黨江啓臣要為造謠行為背書嗎再紅也沒用日動畫師錢比超商員工少支付寶微信支付入列川普禁款中國大陸軟體人妻逛老街問老菜脯價格店家狠嗆不買就滾去吃加色素的華藝疑違法配合陸方審查論文教育部將專案調查億點閱女神降臨有臉蛋天才未演先轟動影綠鬣蜥過馬路彰化網友驚呼蔬菜產區淪陷了嗎影基隆封溪年移除種外來魚類共尾冬天必吃全聯神雞湯讓她整鍋嗑光網讚都是大塊腿肉首例英國變種病毒入侵法國自倫敦返法無症狀隔離中寧夏古早味征服饕客千金傳承美味防萊豬進校園嘉市執行防萊步驟黃敏惠帶校長觀摩火爆哥騎車踹一腳代價萬元又涉恐嚇歲方志友帶球嫁入楊家第一天被公公舉動惹淚了川普支持者闖國會爆衝突當局證實一警傷重不治侯友宜點名永和大陳年內開工都更拚明年核准件莫斯科芭蕾舞團全部停演歲翁撞萬豪車嚇怎麼賠群聚病例增加遭多州禁入澳洲大城雪梨陷孤立氣溫斷崖式崩跌冷爆網揭保暖物松阪蘇馬卡茸薑母鴨政壇大咖該補修的庶民學分爸做忌沒人煮菜女友菜湯感動網衛福部拍板健保費調漲蔡英文集結力量給醫護最大支持台幣強升出口導向企業月賠萬憂倒閉潮房子銀子孩子都被妻掌握科技男提離婚外遇竟曝光凌晨點揪男同學在客廳吃泡麵女兒超扯解釋讓爸起疑病毒變種英國變種病毒流竄德國也現蹤跡小米終於登台接下來還會引進入門款古斌憋氣到缺氧瀕死演技好真實家裡舊車換成新買的進口車他曝曖昧女反差嘴臉網勸放生中共黨員新權利條例海外學者習加強掌控全黨球星不雅片內幕驚爆職棒也受害球星不雅片風暴擴大人遭偷拍山寨新光騙兩岸免息借款千萬真好康他們奸計讓父子奉上間房生命橋梁助學計畫年贊助學子邁向精彩人生免擔心親親口罩公布檢測報告請安心配戴解讀紐交所停止摘牌中國三大電信商馬克宏確診妻子與多位歐洲首長自我隔離財產申報陳菊出任監察院長繳清近千萬房貸憤恨難平他點三明治被誇張加料網得罪外送員台南駕駛超失控連撞車號誌箱整台翻覆人受傷日本單日確診進緊急狀態仍開放台灣等國商務入境人妻衰遇狼密醫免費診療藏色念狼密醫猥褻人妻遭小孩撞見名主播未成年女兒賣淫清純妹變紅牌哭訴爸媽早不管死活股神亞馬遜三方事業關

In [ ]:
count = {}

In [ ]:
for char in egg:
    if char in count.keys():
        count[char] += 1
    else:
        count[char] = 1   

In [ ]:
count

{'男': 555,
 '偶': 27,
 '像': 102,
 '自': 420,
 '爆': 382,
 '出': 788,
 '道': 242,
 '之': 188,
 '後': 428,
 '就': 252,
 '沒': 341,
 '談': 96,
 '過': 307,
 '戀': 72,
 '愛': 292,
 '理': 247,
 '想': 227,
 '型': 136,
 '是': 521,
 '寄': 5,
 '生': 864,
 '上': 722,
 '流': 260,
 '的': 762,
 '她': 230,
 '女': 944,
 '大': 1702,
 '遭': 751,
 '同': 273,
 '學': 647,
 '打': 373,
 '手': 505,
 '槍': 91,
 '射': 22,
 '在': 330,
 '身': 321,
 '校': 300,
 '方': 235,
 '竟': 168,
 '要': 393,
 '當': 279,
 '作': 280,
 '發': 552,
 '新': 1325,
 '加': 290,
 '坡': 13,
 '外': 465,
 '籍': 103,
 '勞': 92,
 '工': 428,
 '已': 99,
 '萬': 658,
 '染': 140,
 '疫': 639,
 '遠': 89,
 '超': 396,
 '本': 306,
 '國': 1484,
 '公': 583,
 '民': 468,
 '年': 1432,
 '不': 1468,
 '給': 118,
 '碰': 15,
 '老': 343,
 '偷': 151,
 '吃': 250,
 '人': 1815,
 '妻': 213,
 '網': 820,
 '挺': 74,
 '被': 635,
 '妳': 39,
 '逼': 76,
 '偏': 35,
 '鄉': 81,
 '和': 158,
 '順': 29,
 '小': 616,
 '童': 89,
 '走': 165,
 '讀': 60,
 '結': 206,
 '合': 251,
 '棒': 69,
 '積': 73,
 '木': 56,
 '飯': 68,
 '店': 311,
 '漂': 17,
 '書': 150,
 '啟': 129,
 '動': 5

### 04. Give each word a number

Sort the values ​​in the previous part, and give each word a number. (words that appear more often rank higher.)



In [ ]:
sorted(count, key=count.get,  reverse=True)

['人',
 '大',
 '台',
 '國',
 '不',
 '年',
 '中',
 '新',
 '一',
 '女',
 '生',
 '網',
 '出',
 '的',
 '遭',
 '家',
 '高',
 '美',
 '上',
 '會',
 '開',
 '萬',
 '有',
 '學',
 '疫',
 '被',
 '全',
 '日',
 '小',
 '北',
 '機',
 '了',
 '成',
 '天',
 '心',
 '友',
 '公',
 '市',
 '車',
 '金',
 '男',
 '下',
 '發',
 '員',
 '到',
 '動',
 '是',
 '長',
 '最',
 '手',
 '子',
 '民',
 '外',
 '曝',
 '元',
 '來',
 '師',
 '法',
 '光',
 '文',
 '後',
 '工',
 '入',
 '分',
 '行',
 '自',
 '防',
 '他',
 '回',
 '為',
 '時',
 '好',
 '月',
 '場',
 '超',
 '主',
 '多',
 '要',
 '名',
 '情',
 '爆',
 '明',
 '前',
 '打',
 '灣',
 '部',
 '變',
 '再',
 '影',
 '用',
 '點',
 '送',
 '警',
 '這',
 '還',
 '南',
 '看',
 '老',
 '沒',
 '重',
 '教',
 '度',
 '電',
 '性',
 '在',
 '起',
 '豬',
 '醫',
 '身',
 '口',
 '業',
 '王',
 '首',
 '登',
 '店',
 '死',
 '過',
 '本',
 '事',
 '進',
 '關',
 '我',
 '個',
 '校',
 '英',
 '案',
 '選',
 '門',
 '愛',
 '病',
 '力',
 '現',
 '股',
 '地',
 '無',
 '加',
 '將',
 '政',
 '能',
 '對',
 '議',
 '驚',
 '作',
 '當',
 '體',
 '院',
 '軍',
 '與',
 '路',
 '強',
 '檢',
 '同',
 '神',
 '傳',
 '球',
 '交',
 '內',
 '戰',
 '三',
 '可',
 '者',
 '萊',
 '資',
 '頭',
 '保',
 '推',
 '都'

Use the `enumerate` function to give each word number from 1 to 3463 in order.

In [ ]:
egg = sorted(count, key=count.get,  reverse=True)

In [ ]:
for i, char in enumerate(egg, 1):
    print(char, i)

人 1
大 2
台 3
國 4
不 5
年 6
中 7
新 8
一 9
女 10
生 11
網 12
出 13
的 14
遭 15
家 16
高 17
美 18
上 19
會 20
開 21
萬 22
有 23
學 24
疫 25
被 26
全 27
日 28
小 29
北 30
機 31
了 32
成 33
天 34
心 35
友 36
公 37
市 38
車 39
金 40
男 41
下 42
發 43
員 44
到 45
動 46
是 47
長 48
最 49
手 50
子 51
民 52
外 53
曝 54
元 55
來 56
師 57
法 58
光 59
文 60
後 61
工 62
入 63
分 64
行 65
自 66
防 67
他 68
回 69
為 70
時 71
好 72
月 73
場 74
超 75
主 76
多 77
要 78
名 79
情 80
爆 81
明 82
前 83
打 84
灣 85
部 86
變 87
再 88
影 89
用 90
點 91
送 92
警 93
這 94
還 95
南 96
看 97
老 98
沒 99
重 100
教 101
度 102
電 103
性 104
在 105
起 106
豬 107
醫 108
身 109
口 110
業 111
王 112
首 113
登 114
店 115
死 116
過 117
本 118
事 119
進 120
關 121
我 122
個 123
校 124
英 125
案 126
選 127
門 128
愛 129
病 130
力 131
現 132
股 133
地 134
無 135
加 136
將 137
政 138
能 139
對 140
議 141
驚 142
作 143
當 144
體 145
院 146
軍 147
與 148
路 149
強 150
檢 151
同 152
神 153
傳 154
球 155
交 156
內 157
戰 158
三 159
可 160
者 161
萊 162
資 163
頭 164
保 165
推 166
都 167
流 168
安 169
連 170
就 171
創 172
合 173
吃 174
真 175
運 176
物 177
理 178
數 179
跨 180
專 181
海 182
水 183
單 184
定 18

Define the sorting result, and set it as a dictionary.

In [ ]:
sorted_char = {char: i for i, char in enumerate(egg, 1)}

Set the blank space as 0.

In [ ]:
sorted_char[" "] = 0

### 05. Encode all sentences

1.   Define a fuction that could make each word turn out to a code.(`char --> sorted_char[char]`)
2.   Use the `map` function to apply on all sentences.


In [ ]:
x = []

for review in x_tmp:
    record = list(map(lambda char:sorted_char[char], review))
    x.append(record)

In [ ]:
y = df["是否適宜觀看"].values

In [ ]:
y

array([1, 0, 1, ..., 1, 1, 1])

### 06. Divide data into "train data" and "test data".

After testing, the ratio of 8:2 is the best training result.

In [ ]:
x_train = x[0:8000]

In [ ]:
y_train = y[0:8000]

In [ ]:
x_test = x[8000:10164]

In [ ]:
y_test = y[8000:10164]

In [ ]:
x_train = sequence.pad_sequences(x_train, maxlen=30)
x_test = sequence.pad_sequences(x_test, maxlen=30)

### 07. Build a deep learning model

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(10000, 256))

In [ ]:
model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))

In [ ]:
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 300)               668400    
_________________________________________________________________
dense (Dense)                (None, 1)                 301       
Total params: 3,228,701
Trainable params: 3,228,701
Non-trainable params: 0
_________________________________________________________________


In the raw data, the ratio of "0" and "1" is 1:9. In order to get a better training result, balance the ratio of "0" and "1" by elevating the weight of "0".

In [ ]:
from sklearn.utils import class_weight
print(np.unique(y_train))
weights = class_weight.compute_class_weight("balanced",np.unique(y_train),y_train)
weights = {i:weights[i] for i in range(2)}
print(weights)

[0 1]
{0: 4.778972520908005, 1: 0.5584252408208851}


### 08. Train the RNN model

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=5,
         validation_data=(x_test, y_test))

Epoch 1/5
250/250 [==============================] - 114s 445ms/step - loss: 0.2876 - accuracy: 0.9063 - val_loss: 0.1516 - val_accuracy: 0.9445
Epoch 2/5
250/250 [==============================] - 99s 394ms/step - loss: 0.1132 - accuracy: 0.9592 - val_loss: 0.1608 - val_accuracy: 0.9543
Epoch 3/5
250/250 [==============================] - 98s 392ms/step - loss: 0.0820 - accuracy: 0.9739 - val_loss: 0.1395 - val_accuracy: 0.9570
Epoch 4/5
250/250 [==============================] - 99s 396ms/step - loss: 0.0577 - accuracy: 0.9814 - val_loss: 0.1445 - val_accuracy: 0.9506
Epoch 5/5
250/250 [==============================] - 100s 401ms/step - loss: 0.0510 - accuracy: 0.9829 - val_loss: 0.2366 - val_accuracy: 0.9529


### 09. Test and Application

In [ ]:
def predict(input):
  encode_input = list(map(lambda char:sorted_char[char],' '.join(patn.findall(input))))
  output = model.predict([encode_input])[0][0]
  print('Predicted probability: {}'.format(output))
  print('Prediction: {}'.format('banned' if output < 0.5 else 'passed'))

In [ ]:
predict('少女爛醉認錯男友！嘿咻2次還想要　他躲室友房、她裸身拍門狂求歡酒醒嚇瘋')

Predicted probability: 0.017499208450317383
Prediction: banned


In [ ]:
predict('【獨家／醫師染疫1】首例確診醫師遭爆在院內未戴口罩　護理師女友同住中招')

Predicted probability: 0.9967807531356812
Prediction: passed


In [ ]:
predict('13歲女5月4度激戰3男　鷹眼媽靠房內一瓶礦泉水揪出真相')

Predicted probability: 0.17591020464897156
Prediction: banned


In [ ]:
predict('台中夜店流出「全裸女」3P片！開幕主打高爾宣　鹹濕試營運遭警嚴辦')

Predicted probability: 0.17476296424865723
Prediction: banned
